In [49]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/hojjatk/mnist-dataset")

Skipping, found downloaded files in "./mnist-dataset" (use force=True to force download)


In [50]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from torchsummary import summary
import gzip

device =  "cuda" if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [51]:
def read_idx_images(filename):
    with open(filename, 'rb') as f:   # <-- normal binary open
        data = f.read()
        
    magic = int.from_bytes(data[0:4], 'big')
    num_images = int.from_bytes(data[4:8], 'big')
    rows = int.from_bytes(data[8:12], 'big')
    cols = int.from_bytes(data[12:16], 'big')
    
    images = np.frombuffer(data, dtype=np.uint8, offset=16)
    images = images.reshape(num_images, 1, rows, cols).astype(np.float32) / 255.0
    return images

def read_idx_labels(filename):
    with open(filename, 'rb') as f:   # <-- normal binary open
        data = f.read()
        
    magic = int.from_bytes(data[0:4], 'big')
    num_labels = int.from_bytes(data[4:8], 'big')
    
    labels = np.frombuffer(data, dtype=np.uint8, offset=8)
    return labels


In [52]:
class DataSet(Dataset):
    def __init__(self, images , labels):
        self.images = torch.tensor(images, dtype = torch.float32)
        self.labels = torch.tensor(labels, dtype = torch.float32)
        
    def __len__(self):
        return(len(self.images))
    
    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

In [53]:
test_Ipath=r"/home/namankarki/Naman/3_months_of_data-science/Deep_Learning_Pytorch/cnn/mnist-dataset/t10k-images-idx3-ubyte/t10k-images-idx3-ubyte"
test_Lpath=r"/home/namankarki/Naman/3_months_of_data-science/Deep_Learning_Pytorch/cnn/mnist-dataset/t10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte"
train_Ipath=r"/home/namankarki/Naman/3_months_of_data-science/Deep_Learning_Pytorch/cnn/mnist-dataset/train-images-idx3-ubyte/train-images-idx3-ubyte"
train_Lpath=r="/home/namankarki/Naman/3_months_of_data-science/Deep_Learning_Pytorch/cnn/mnist-dataset/train-labels-idx1-ubyte/train-labels-idx1-ubyte"



test_images = read_idx_images(test_Ipath)
test_labels = read_idx_labels(test_Lpath)
train_images = read_idx_images(train_Ipath)
train_labels = read_idx_labels(train_Lpath)


In [54]:
train_dataset = DataSet(train_images, train_labels)
test_dataset = DataSet(test_images, test_labels)

In [55]:
train_dataloader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
test_dataloader = DataLoader(train_dataset, batch_size = 100, shuffle = False)

In [56]:
data=iter(train_dataloader)
image,label = next(data)
print(image.shape, label.shape)


torch.Size([64, 1, 28, 28]) torch.Size([64])


In [57]:
class MyModel(nn.Module):
    def __init__(self):
        super.__init__()
        